# Network and relations making

In [1]:
import subprocess
import os
import time
from yml_create_functions import make_yml, make_network_agents_yml

In [2]:
name_simulation = 'simulation_tutorial'

## Network Agent Parameters

The schema can be changed in schema/schema.py. Here, you can change the parameters of the default schema

In [17]:
network_agents_parameters = {
    "default": {
        "id_message": "NaN",
        "has_tv": "false",
        "cause": -1,
        "method": "NaN",

        "type": "dumb",
        "response": "NaN",
        "stance": "agree",
        "repost": "NaN",
        "parent_id": "NaN"

    },

    "DumbViewer": [
        {"weight": 2, "type": "dumb"},
        {"weight": 2, "type": "dumb", "has_tv": "true"}
    ],
    "HerdViewer": [
        {"weight": 2, "type": "herd", "stance": "against"},
        {"weight": 2, "type": "herd", "has_tv": "true"}
    ],
    "WiseViewer": [
        {"weight": 1, "type": "wise", "stance": "against"},
        {"weight": 1, "type": "wise", "has_tv": "true", "stance": "against"}
    ],

}

## Other Parameters

In [18]:
parameters = {
    "default_state": "{}",
    "load_module": "schema", # path from yml to schema
    "environment_agents": "[]",
    "environment_class": "schema.NewsEnvironmentAgent", # path from yml to environment class
    "environment_params": {
        "prob_neighbor_spread": 0.02,  # 006
        "prob_tv_spread": 0.006,  # 001
        "prob_neighbor_cure": 0.006,  # 001
        "prob_backsliding": 0.01,  # 003
        "prob_dead": 0.006,  # 001,
        "prob_repost": 0.8,
        "hostility_weight": 1,  # Unused yet
        "mean_time_connection": 50,
        "var_time_connection": 30
    },
    "interval": 1,
    "max_time": 100,
    "name": name_simulation,
    "network_params": {
        "generator": "barabasi_albert_graph",
        "n": 100,
        "m": 5
    },
    "num_trials": 1
}

# Prob responses parameters

In [19]:
# Define your probabilities here
prob_response = {"dumb": {"support": 0.4, "deny": 0.3, "question": 0, "comment": 0.3},
                "herd": {"support": 0.25, "deny": 0.25, "question": 0.25, "comment": 0.25},
                "wise": {"support": 0.2, "deny": 0.2, "question": 0.3, "comment": 0.3}
                }
types = list(prob_response.keys())
responses = list( prob_response[types[0]].keys() )

for type in types:
    for response in responses:
        name_i = f"prob_{type}_{response}"
        prob_i = prob_response[type][response]
        # Here, we set each probability
        parameters["environment_params"][name_i] = prob_i

## Simulation execution

In [20]:
parameters_i = parameters.copy()
network_agents_parameters_i = network_agents_parameters.copy()
data = make_yml(parameters_i)
data += make_network_agents_yml(network_agents_parameters_i)

yml_path = os.path.join('schema', '{name_simulation}.yml') # YML path
with open(yml_path, 'w') as file:
    file.write(data)

command = "soil"
start = time.time()

output = subprocess.check_output([command, yml_path])
end = time.time()
seconds_simulation = str(end-start)
print("SIMULATION'S SECONDS:", seconds_simulation)

SIMULATION'S SECONDS: 6.912971258163452


# Conversation Making

In [3]:
from get_data.get_data import get_pivoted_data
from post.templates import POST_TEMPLATE, REPLY_TEMPLATE, INSTRUCTIONS_TEMPLATE
from anytree import Node, RenderTree, LevelOrderIter
from post.post import Post
from IA.ia import send_prompt

## Output sqlite path

In [4]:
analysis_path = os.path.join('soil_output', name_simulation)
sql_table_path = f'{name_simulation}_trial_0.sqlite'

## Get Data

In [5]:
data = get_pivoted_data(analysis_path, sql_table_path)

## Define News

In [6]:
NEWS = 'Chilean Government Announces Ambitious Plan for Sustainable Energy Transition'
SHOW_TREE = True

## Create the Network's Tree

In [12]:
root = Post(0, message=NEWS, post_template=POST_TEMPLATE,
             reply_template=REPLY_TEMPLATE,
             instructions_template=INSTRUCTIONS_TEMPLATE)
lista_nodos = [root] + [Post('', message='Empty') for _ in range(len(data))]

In [13]:
for i in range(len(data)):
    aux = data[i]
    owner = aux[0]
    cause = aux[3]
    method = aux[4]
    response = aux[5]
    stance = aux[6]
    repost = aux[7]
    if repost == 'true':
        repost = True
    elif repost == 'false':
        repost = False
    else:
        print(data[i])
        print(repost)
        raise ValueError("Error repost format")
    
    name = aux[8]
    parent = lista_nodos[int(aux[9])]

    message = ''

    lista_nodos[i+1] = Post(name, parent=parent, owner=owner,
                             message=message, stance=stance,
                             repost=repost, cause=cause,
                             method=method, response=response,
                             post_template=POST_TEMPLATE,
                             reply_template=REPLY_TEMPLATE,
                             instructions_template=INSTRUCTIONS_TEMPLATE,
                             news=NEWS)

## Associate a Written Post to each Message

In [14]:
from parameters import API_KEY
import openai
openai.api_key = API_KEY

In [15]:
list_messages = []
for node_i in LevelOrderIter(root):
    if node_i.repost or node_i == root:
        continue

    instructions, prompt = node_i.get_prompt(language='english', 
                min_caract=130, max_caract=250,
                user_description='average toxic and angry social media user')
    
    answer = send_prompt(instructions, prompt, temp=1.5, max_tokens=1000)
    
    node_i.set_message(answer)
    list_messages.append((node_i, prompt))

## Plot the Network in a Tree

In [16]:
if SHOW_TREE:
    for pre, _, node in RenderTree(root):
        if node.repost:
            message = "repost"
        else:
            message = "'"+node.message+"'"
        
        if node.name == 0:
            print(f"{pre}NEWS: {message}")
        else:
            print(f"{pre}{node.owner} {message}")

NEWS: 'Chilean Government Announces Ambitious Plan for Sustainable Energy Transition'
├── 43 'Awesome news! 🎉🌍 Chile's government has just revealed their ambitious plan for a sustainable energy transition! 👏🔋 They're committed to slashing CO2 emissions and pursuing clean energy sources 🍃💨 It's a major step towards fighting climate change and reducing environmental impact. 🌱💚 Way to go, Chile! 🇨🇱 Let's inspire others to take big strides in sustainability too! 👊😎 #sustainablefuture #cleanenergy #climateaction'
│   ├── 50 'Whoa! Check out Chile's new plan for sustainable energy transition! 💥💪 They're getting serious about reducing CO2 emissions and promoting clean energy sources. 🍃🔌 Talk about stepping up their game! 🙌🌱 Let's give them props for advancing the fight against climate change. 🌍🔥 Other countries, are you watching? Take a cue from Chile and usher in a #sustainablefuture! 👊😎 #cleanenergy #climateaction #goals'
│   └── 24 'That's really cool! 😍🌍 Chile surely knows how to impress!